In [19]:
import sys
sys.path.append('..')
from khan_project_data import *


In [20]:
def self_git(ProjectID):
    """
    Purpose:获取项目内代码库 
    """
    conn = pymysql.connect(**conn_khan)
    cursor = conn.cursor()
    project_git_list = '''  
        SELECT
            PV.project_id '项目ID',
            PV.id '版本ID',
            PR.repository_id '代码库ID',
            GITR.`name`'代码库名称' 
	
        FROM
            kh_project_version PV 
            LEFT JOIN kh_project__repository PR ON PV.project_id = PR.project_id
            LEFT JOIN kh_gitlab_repository GITR ON GITR.id = PR.repository_id

        WHERE
	        pv.project_id = {0}'''.format(ProjectID)
    
    cursor.execute(project_git_list)
    result = cursor.fetchall()
    # 关闭光标对象
    cursor.close()
    conn.close()
    return result
    
# end def

In [21]:
def depend_git(ProjectID):
    """
    Purpose:获取项目内代码库 
    """
    conn = pymysql.connect(**conn_khan)
    cursor = conn.cursor()
    project_git_list = '''  
        SELECT
            PV.project_id '项目ID',
            PV.id '版本ID',
            PVD.depend_project_id '依赖产品ID',
            P.name '依赖产品名称',
            PR.repository_id '代码库ID',
            GITR.`name`'代码库名称' 
            
        FROM
            kh_project_version PV 
            LEFT JOIN kh_project_version_dependency PVD ON PVD.project_version_id = PV.id
            LEFT JOIN kh_project__repository PR ON PVD.depend_project_id = PR.project_id
            LEFT JOIN kh_gitlab_repository GITR ON GITR.id = PR.repository_id
            LEFT JOIN kh_project p ON PVD.depend_project_id = p.id 

        WHERE
            pv.project_id = {0}'''.format(ProjectID)
    
    cursor.execute(project_git_list)
    result = cursor.fetchall()
    # 关闭光标对象
    cursor.close()
    conn.close()
    return result
    
# end def

In [22]:
pid = 1348
print(self_git(pid))

OperationalError: (2003, "Can't connect to MySQL server on '192.168.95.170' (timed out)")

In [ ]:
print(depend_git(pid),sep='\n') 

OperationalError: (2003, "Can't connect to MySQL server on '192.168.95.170' (timed out)")